<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
import pandas as pd
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [120]:
pd.options.display.max_colwidth = 1000

In [121]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

data = pd.read_csv(csv_file)
data.head(1)

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...,CLOSED,FIXED,2,10/10/2001 21:34,2/9/2012 15:57


In [122]:
data = data.drop(columns=['Priority', 'Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
data.head(1)

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...


In [123]:
data = data.dropna(axis=0, subset=['Description'])
data.head(1)

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...


In [124]:
#Removing the fixed patterns by replacing them with " "
data["Description"]= data["Description"].str.replace("fixed in HEAD", "", case = False)
data["Description"]= data["Description"].str.replace("has been marked as readonly", " ", case = False)

In [125]:
data[data['Description'].map(len) > 2]

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...
1,2,NaN,Opening repository resources doesnt honor type (1GE6I1Q),Opening repository resource always open the default text editor and doesnt; \thonor any mapping between resource types and editors.; \tAs a result it is not possible to view the contents of an image (*.gif file) in a sensible way.; ; NOTES:; EG (5/24/2001 5:13:45 AM); \tAgreed that this is needed eventually but not critical for June. \t; \tThere should also be a Java outliner when editing a java resource from the repository.; \tMoving to VCM.; ; KM (5/24/01 2:09:21 PM); \tIn general you cannot open a specialized editor on a remote resource because they can sometimes; \tbe dependent on other resources and expect these to be local. For example; an HTML page which; \treferences a graphic; the HTML editor is going to try to look for the graphic to display it.; ; \tYou cant be sure that the editor was written to accomodate remote or missing resources.; \tWe discussed this some time ago with Dave T. and felt it was safer to just use vanilla editors for now.; ; KM (5/24/01 4:42:09 PM); \...
2,3,NaN,Sync does not indicate deletion (1GIEN83),KM (10/2/2001 5:55:18 PM); \tThis PR about the deletion indicator in the sync viewer being too subtle.; \tThis would be even more true for someone with vision problems.; \tNE has good suggestion of using the label of the compare text viewer.; \tWe may also consider a better icon.; \tOn to the PR....; ; - delete file from project; - synchronize; - there is no indication that the file has been deleted from the workspace; - it looks the same as if the file is in the workspace but is empty; ; NOTES:; ; KM (8/15/2001 3:37:53 PM); \tIt should appear as a rightfacing - arrow in the sync view.; \tSent screen cap to NE requesting clarification.; ; NE (8/20/01 9:13:38 AM); \tThe - in the arrow is clearer when there are other arrows present too (the + and change ones).; \tOtherwise; its easy to miss the distinction; especially when the users attention is focussed; \ton the text compare panes.; \tId expect the Workspace file: label to indicate that it has been deleted.; \tThe Repository file: ...
3,4,NaN,need better error message if catching up over read-only resource (1GF69TF),- become synchronized with some project in a repository; - use a different Eclipse to make a change to a file resource within this project and release it to the repository; - in the original Eclipse mark this file resource as being read-only (select it; right-click -> Properties; change; OK); - select the file resource; right-click -> Team -> Synchronize with Stream; - in the subsequent comparison view select the file resource; right-click -> Catchup; - since it has been marked as read-only there are inevitable problems. However the error dialog that is shown; \tdoes not offer much assistance (An IO error occurred: IO Error); ; ; NOTES:
4,5,NaN,ISharingManager sharing API inconsistent (1GAUL8H

In [126]:
import re
import string

def clean_text_from_description_title(text):

    text = text.replace('\t', '')

    text = re.sub('\w*\d\w*', '', text)
    text = text.lower()

    text = text.translate(str.maketrans('', '', string.punctuation))
    
    return text

step_1 = lambda x: clean_text_from_description_title(x)
step_2 = lambda x: clean_text_from_description_title(x)

data['Description'] = data['Description'].apply(step_1)
data['Title'] = data['Title'].apply(step_2)
data.loc[[0]]

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,usability issue with external editors,setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with an external editor found the modified file on disk and showed the changes the real problem occurs if release actually finds something to release but you dont spot that some resources are missing this is extremely error prone one of my changes didnt made it into build because of this notes eg pm release should do a refresh from local before doing the release moving to vcm km pm comments from jm in related email should not do this for free could have a setting which made it optoinal but...


In [127]:

def removalWord_tokenization_lemmatization(text):
    result = []

    for token in SIMPLE_PREPROCESS(text):
        if token not in STOP_WORDS  and len(token) > 5:
            result.append(WordNetLemmatizer().lemmatize(token, pos='v'))
            
    return result


data['Title'] = data['Title'].map(removalWord_tokenization_lemmatization)
data['Description'] = data['Description'].map(removalWord_tokenization_lemmatization)

data.loc[[0]]

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [128]:
duplicated_reports = data.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.head(1)

,Issue_id,Duplicated_issue,Title,Description
0,25,28126.0,"[default, connection]","[helpful, notion, default, repository, connection, connect, project, wouldnt, everytime, create, create, default, minimum, have, default, repository, helpful, pulldown, connection, wizard, default]"


In [129]:
master_reports = data[data.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.head(1)

,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [130]:
#Creating a dictionary using gensim library
#the dictionary object is used to create a bag of words (BoW) corpus which further used as the input to topic modelling and other models as well

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 500:
        break


0 actually
1 builders
2 change
3 comment
4 compare
5 complete
6 consider
7 consistency
8 contain
9 default
10 derive
11 document
12 drive
13 editor
14 effect
15 ensure
16 external
17 extremely
18 fail
19 inconsistent
20 internal
21 maintain
22 mandatory
23 miss
24 modify
25 move
26 navigator
27 network
28 occur
29 open
30 optimal
31 problem
32 project
33 refresh
34 relate
35 release
36 resource
37 resources
38 responsible
39 runnable
40 set
41 show
42 summary
43 verify
44 workspace
45 workspaces
46 accomodate
47 agree
48 content
49 critical
50 dependent
51 discuss
52 display
53 doesnt
54 edit
55 editors
56 eventually
57 example
58 expect
59 general
60 graphic
61 map
62 need
63 outliner
64 possible
65 reference
66 remote
67 repositories
68 repository
69 result
70 sensible
71 specialize
72 vanilla
73 write
74 appear
75 arrows
76 attention
77 better
78 clarification
79 clearer
80 correctly
81 delete
82 deletion
83 distinction
84 especially
85 focus
86 indicate
87 indication
88 indicator
8

In [131]:
#Creating BoW using the the dictionary
#it is the corpus that contains the word id and its frequency in each document

bag_of_words_corpus = [dictionary.doc2bow(doc) for doc in master_reports['Description']]
bag_of_words_corpus[0]

[(0, 1),
 (1, 2),
 (2, 5),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 3),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 3),
 (33, 3),
 (34, 1),
 (35, 12),
 (36, 5),
 (37, 2),
 (38, 1),
 (39, 2),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 2),
 (44, 2),
 (45, 1)]

In [132]:
#Printing the BoW for single document
bag_of_words_doc = bag_of_words_corpus[0]
for i in range(len(bag_of_words_doc)):
    print("{}. (\"{}\") word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

1. ("actually") word seen 1 time.
2. ("builders") word seen 2 time.
3. ("change") word seen 5 time.
4. ("comment") word seen 1 time.
5. ("compare") word seen 1 time.
6. ("complete") word seen 1 time.
7. ("consider") word seen 1 time.
8. ("consistency") word seen 1 time.
9. ("contain") word seen 1 time.
10. ("default") word seen 1 time.
11. ("derive") word seen 1 time.
12. ("document") word seen 1 time.
13. ("drive") word seen 1 time.
14. ("editor") word seen 3 time.
15. ("effect") word seen 1 time.
16. ("ensure") word seen 1 time.
17. ("external") word seen 3 time.
18. ("extremely") word seen 1 time.
19. ("fail") word seen 1 time.
20. ("inconsistent") word seen 1 time.
21. ("internal") word seen 1 time.
22. ("maintain") word seen 2 time.
23. ("mandatory") word seen 1 time.
24. ("miss") word seen 1 time.
25. ("modify") word seen 1 time.
26. ("move") word seen 1 time.
27. ("navigator") word seen 2 time.
28. ("network") word seen 1 time.
29. ("occur") word seen 1 time.
30. ("open") word s

In [133]:
corpus = bag_of_words_corpus

#Training the LDA model on the BoW corpus
lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

In [134]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.069*"public" + 0.066*"import" + 0.056*"display" + 0.024*"static" + 0.018*"snippet" + 0.016*"string" + 0.016*"column" + 0.013*"control" + 0.013*"swtnone" + 0.013*"private"
Topic: 1 
Words: 0.046*"javathread" + 0.045*"daemon" + 0.038*"threadblocked" + 0.032*"console" + 0.024*"breakpoint" + 0.020*"variable" + 0.019*"variables" + 0.019*"memory" + 0.017*"symbol" + 0.013*"report"
Topic: 2 
Words: 0.104*"project" + 0.034*"create" + 0.028*"workspace" + 0.027*"change" + 0.019*"delete" + 0.018*"folder" + 0.016*"resource" + 0.013*"select" + 0.013*"repository" + 0.012*"eclipse"
Topic: 3 
Words: 0.124*"method" + 0.112*"thread" + 0.044*"object" + 0.035*"wait" + 0.034*"available" + 0.034*"native" + 0.032*"lock" + 0.030*"monitor" + 0.028*"runnable" + 0.027*"cwindowsdll"
Topic: 4 
Words: 0.037*"select" + 0.036*"dialog" + 0.028*"button" + 0.022*"reproduce" + 0.019*"search" + 0.017*"eclipse" + 0.016*"launch" + 0.013*"result" + 0.013*"problem" + 0.013*"windows"
Topic: 5 
Words: 0.109*"e

In [135]:
print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -6.544719266606948


In [136]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=data['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.6130179406184734


In [111]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=665f7cac88fc071a278f48ea3c43e822fe200d5c96e279550507316cade67932
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=f38e70b1e1b6585ba43f5a9156c9742ea261ea8774694868b05bca869f521424
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


In [112]:
import pyLDAvis
import pyLDAvis.gensim_models # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [113]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.180433 -0.019897       1        1  16.631732
2      0.205035  0.050165       2        1  15.875922
7      0.107867  0.077116       3        1  15.219781
6      0.186229  0.011115       4        1  13.648737
0      0.023662  0.020800       5        1  10.032604
3     -0.147874  0.050497       6        1   8.041412
8      0.105739 -0.043882       7        1   6.750586
9     -0.336610  0.109386       8        1   5.360864
4     -0.145557 -0.429399       9        1   4.231454
5     -0.178924  0.174099      10        1   4.206909, topic_info=        Term          Freq         Total Category  logprob  loglift
32   project  22459.000000  22459.000000  Default  30.0000  30.0000
372   source  16250.000000  16250.000000  Default  29.0000  29.0000
116   method  19700.000000  19700.000000  Default  28.0000  28.0000
589  message  13283.000000  13283.000000  Default  27.0000  27.0000
103  eclipse  29432.000000  29432.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
355   native    490.903768   2528.425262  Topic10  -4.9254   1.5293
28     occur    544.759743   4880.428841  Topic10  -4.8213   0.9758
224   follow    604.358910   9778.158990  Topic10  -4.7175   0.3847
786  windows    508.846754   7250.652523  Topic10  -4.8895   0.5117
698  browser    413.983927   2966.887533  Topic10  -5.0959   1.1990

[628 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
215       1  0.970055      ability
215       3  0.027962      ability
215       4  0.002151      ability
819       7  0.996593  accelerator
2555      6  0.995788      acquire
...     ...       ...          ...
44        3  0.109140    workspace
44        4  0.794655    workspace
44        6  0.090909    workspace
44        8  0.005174    workspace
4490      3  0.998369    xulrunner

[1187 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 8, 7, 1, 4, 9, 10, 5, 6])

In [ ]:
#Creating 10 empty clusters and pushing master reports in each of them based on topic modeling and saving them in individual csv file
import pandas as pd

for c in range(10):
    exec('topic_{} = pd.DataFrame()'.format(c))

    for i in range(len(master_reports)):
        topic=lda_model[dictionary.doc2bow(master_reports.Description[i])]
        topic= np.asarray(topic)
        
        if int(topic[np.argmax(topic[:,1]),0])== c:
            exec('topic_{} = topic_{}.append(master_reports.loc[[i]])'.format(c,c))
            exec('topic_{} = topic_{}.reset_index(drop=True)'.format(c,c))
            exec('topic_{}.to_csv("topic_{}.csv")'.format(c,c))